In [1]:
from time import sleep
import yaml
from yaml.loader import SafeLoader
import logging

logging.basicConfig(filename='WorkFlowLog.txt', filemode='w',
                    format='%(asctime)s;%(message)s', datefmt='%Y-%m-%d %H:%M:%S.000000', level=logging.INFO)


def TimeFunction(tName, duration):
    sleep(int(duration))


Tasks = {
    'TimeFunction': TimeFunction
}


def executeFlow(name, flow):
    if flow['Execution'] == 'Sequential':
        for key, act in flow['Activities'].items():
            log_name = name + '.' + key
            act_type = act['Type']
            if act_type == 'Flow':
                logging.info('%s Entry', log_name)
                executeFlow(log_name, act)
                logging.info('%s Exit', log_name)
            if act_type == 'Task':
                executeTask(log_name, act)


def executeTask(name, task):
    logging.info('%s Entry', name)
    logging.info('%s Executing %s', name, str.format(
        "{}({},{})", task['Function'], task['Inputs']['FunctionInput'], task['Inputs']['ExecutionTime']))
    Tasks[task['Function']](task['Inputs']['FunctionInput'],
                            task['Inputs']['ExecutionTime'])
    logging.info('%s Exit', name)


with open('Milestone1A.yaml', 'r') as InputFile:
    workflow = list(yaml.load_all(InputFile, Loader=SafeLoader))

    for wf in workflow:
        for key, value in wf.items():
            logging.info('%s Entry', key)
            steps = value
            if(steps['Type'] == 'Flow'):
                executeFlow(key, steps)
            if(steps['Type'] == 'Task'):
                executeTask(key, steps)
            logging.info('%s Exit', key)